In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("datasets_eat_places.csv")

# Знакомство с датасетом

org_id - идентификатор заведения

city - город, в котором находится заведение (msk или spb)

average_bill - средний чек в заведении

rating - рейтинг заведения

rubric - тип заведения

Остальные столбцы - человекочитаемые бинарные фичи заведений

Рандомные строки датасета:

In [3]:
df.sample(5)

,Unnamed: 0,org_id,city,average_bill,rating,rubric,coffee_to_go,food_delivery,breakfast,summer_terrace,...,business_lunch,payment_by_credit_card,car_park,gift_certificate,karaoke,free_delivery,music,pets,air_conditioning,view_on_ostankino_tower
12637,19433,1959759105123419751,msk,NaN,3.750000,Быстрое питание,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
10786,16257,3069795362143174800,msk,NaN,NaN,Ресторан,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1096,2128,12029360603750229027,msk,NaN,NaN,Ресторан,1,1,1,1,...,1,1,0,0,1,0,0,0,0,1
18340,27095,6534417807710254400,spb,500.0,3.750000,"Булочная, пекарня",1,0,1,1,...,1,1,0,1,0,0,0,0,0,0
9195,10618,16038508537077732023,msk,1000.0,4.517241,"Бар, паб",1,1,1,1,...,1,1,0,1,1,0,0,0,0,0


# Работа с пропущенными значениями

метод info(), чтобы увидеть число пропущенных значений

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43195 entries, 0 to 43194
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               43195 non-null  int64  
 1   org_id                   43195 non-null  uint64 
 2   city                     43195 non-null  object 
 3   average_bill             17820 non-null  float64
 4   rating                   36438 non-null  float64
 5   rubric                   43195 non-null  object 
 6   coffee_to_go             43195 non-null  int64  
 7   food_delivery            43195 non-null  int64  
 8   breakfast                43195 non-null  int64  
 9   summer_terrace           43195 non-null  int64  
 10  wi_fi                    43195 non-null  int64  
 11  business_lunch           43195 non-null  int64  
 12  payment_by_credit_card   43195 non-null  int64  
 13  car_park                 43195 non-null  int64  
 14  gift_certificate      

количество пропущенных значений

In [5]:
df.isna().sum() 

Unnamed: 0                     0
org_id                         0
city                           0
average_bill               25375
rating                      6757
rubric                         0
coffee_to_go                   0
food_delivery                  0
breakfast                      0
summer_terrace                 0
wi_fi                          0
business_lunch                 0
payment_by_credit_card         0
car_park                       0
gift_certificate               0
karaoke                        0
free_delivery                  0
music                          0
pets                           0
air_conditioning               0
view_on_ostankino_tower        0
dtype: int64

Очистка датасета от всех заведений, у которых средний чек неизвестен или превышает 2500

In [6]:
df_query_bill = df[(df['average_bill'].isna() == False) & (df['average_bill'] <= 2500)]

проверка:

In [7]:
average_bill_sum_na = df_query_bill['average_bill'].isna().sum()
average_bill_max = df_query_bill['average_bill'].max()
print(f'Количество пропусков фичи average_bill: {average_bill_sum_na}\nМаксимальное значение фичи average_bill: {average_bill_max}')

Количество пропусков фичи average_bill: 0
Максимальное значение фичи average_bill: 2500.0


Заполнение оставшихся пропусков

In [8]:
mean_rating = df['rating'].mean()
df_query_bill = df_query_bill.fillna({'rating': mean_rating})

In [9]:
df = df_query_bill.copy()

# Анализ датасета

Каких заведений больше - из Москвы или Санкт-Петербурга?

In [10]:
city_amount = df.groupby('city')['org_id'].count().to_frame().rename(columns={'org_id': 'amount'})
city_amount

,amount
city,
msk,11075
spb,6262


In [11]:
msk_spb = city_amount.loc['msk', 'amount'] / city_amount.loc['spb', 'amount']
print(f'Заведений в Москве больше в {round(msk_spb, 3)} раз')

Заведений в Москве больше в 1.769 раз


Какие типы заведений есть в датасете?

In [12]:
print(f"Количество типов заведений: {df['rubric'].nunique()}")

Количество типов заведений: 14


In [13]:
df['rubric'].unique()

array(['Быстрое питание', 'Кофейня', 'Столовая', 'Бар, паб',
       'Булочная, пекарня', 'Ресторан', 'Пиццерия', 'Кафе', 'Суши-бар',
       'Кондитерская', 'Кофе с собой', 'Кальян-бар',
       'Бар безалкогольных напитков', 'Спортбар'], dtype=object)

Сколько в датасете суммарно ресторанов и пабов?

In [14]:
amount = len(df.query("(rubric == 'Ресторан') | (rubric == 'Бар, паб')"))
print(f'В датасете суммарно ресторанов и пабов: {amount}')

В датасете суммарно ресторанов и пабов: 5530


Какой у них средний чек?

In [15]:
average_bill = df.query("(rubric == 'Ресторан') | (rubric == 'Бар, паб')")['average_bill'].mean()
print(f'средний чек: {round(average_bill, 3)}')

средний чек: 1099.096


Какая доля заведений “Кофе с собой” действительно имеет фичу кофе на вынос?

In [16]:
coffee_proportion = len(df.query("(rubric == 'Кофе с собой') & (coffee_to_go == 1)")) / len(df.query("rubric == 'Кофе с собой'"))
print(f'Доля заведений “Кофе с собой”, которые действительно имеет фичу кофе на вынос: {round(coffee_proportion, 3)*100}%')

Доля заведений “Кофе с собой”, которые действительно имеет фичу кофе на вынос: 59.0%


Найдите разницу между средним чеком всех кафе в Москве и Санкт-Петербурге

In [17]:
average_bill_by_city = df.groupby('city')['average_bill'].mean().to_frame()
average_bill_by_city

,average_bill
city,
msk,784.288939
spb,689.556052


In [18]:
difference = average_bill_by_city.loc['msk', 'average_bill'] - average_bill_by_city.loc['spb', 'average_bill']
print(f'Средний чек в Москве больше среднего чека в Питере на {round(difference, 2)} рублей')

Средний чек в Москве больше среднего чека в Питере на 94.73 рублей


# Личные наблюдения

In [19]:
df.describe()

,Unnamed: 0,org_id,average_bill,rating,coffee_to_go,food_delivery,breakfast,summer_terrace,wi_fi,business_lunch,payment_by_credit_card,car_park,gift_certificate,karaoke,free_delivery,music,pets,air_conditioning,view_on_ostankino_tower
count,17337.000000,1.733700e+04,17337.000000,17337.000000,17337.000000,17337.000000,17337.000000,17337.000000,17337.000000,17337.000000,17337.000000,17337.000000,17337.000000,17337.000000,17337.000000,17337.000000,17337.000000,17337.000000,17337.000000
mean,43530.963662,9.209979e+18,750.072100,4.140882,0.751745,0.820788,0.599758,0.708312,0.879506,0.721578,0.866182,0.022668,0.099787,0.336044,0.050182,0.038992,0.000750,0.000923,0.211109
std,18231.825285,5.338317e+18,461.268841,0.601254,0.432013,0.383541,0.489961,0.454553,0.325547,0.448235,0.340466,0.148848,0.299724,0.472368,0.218326,0.193581,0.027374,0.030366,0.408107
min,1.000000,5.786926e+13,500.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,33244.000000,4.602787e+18,500.000000,3.933333,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,47492.000000,9.183887e+18,500.000000,4.187500,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,57947.000000,1.381042e+19,1000.000000,4.513138,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,68332.000000,1.844028e+19,2500.000000,5.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [20]:
print(f'Средний рейтинг заведений: {round(df.describe().loc["mean", "rating"], 2)}/5')

Средний рейтинг заведений: 4.14/5


In [21]:
print(f'Доля заведений, имеющих wifi: {round(df.describe().loc["mean", "wi_fi"], 2) * 100}%')
print(f'Их количество: {len(df[df["wi_fi"] == 1])}')

Доля заведений, имеющих wifi: 88.0%
Их количество: 15248


In [22]:
print(f'Доля заведений, где разрешены питомцы: {round(df.describe().loc["mean", "pets"], 6) * 100}%')
print(f'Их количество: {len(df[df["pets"] == 1])}')

Доля заведений, где разрешены питомцы: 0.075%
Их количество: 13


# Заведения Питера

In [23]:
piter_df = df[df['city'] == 'spb']

In [24]:
average_bill_all = df[df['rubric'] == 'Ресторан']['average_bill'].mean()
average_bill_piter = piter_df[piter_df['rubric'] == 'Ресторан']['average_bill'].mean()

In [25]:
print(f'Средний чек уменьшился на {round(average_bill_all - average_bill_piter, 2)} рублей')

Средний чек уменьшился на 157.64 рублей


In [26]:
food_delivery_all = df.groupby('rubric', as_index=False)['food_delivery'].mean().sort_values(by='food_delivery').rename(columns={'food_delivery': 'all_delivery'})
food_delivery_piter = piter_df.groupby('rubric', as_index=False)['food_delivery'].mean().sort_values(by='food_delivery').rename(columns={'food_delivery': 'piter_delivery'})

In [38]:
food_delivery_all.merge(food_delivery_piter)

,rubric,all_delivery,piter_delivery
0,Кондитерская,0.289720,0.386667
1,Бар безалкогольных напитков,0.454545,0.500000
2,Кофе с собой,0.539568,0.560976
3,"Булочная, пекарня",0.621253,0.578947
4,Столовая,0.639437,0.641694
5,Кофейня,0.659786,0.656753
6,Быстрое питание,0.773448,0.765550
7,Кальян-бар,0.857471,0.836207
8,Кафе,0.863740,0.851904
9,Спортбар,0.878788,1.000000


1) Все спортбары Питера имеют доставку в то время, как 87% спортбаров Москвы и Питера имеют доставку

2) Значительно изменился процент кондитерских, которые имеют доставку

In [69]:
proportion_piter = (piter_df.groupby('rubric').size() / len(piter_df)).sort_values().to_frame().reset_index().rename(columns={0: 'piter'})
proportion_all = (df.groupby('rubric').size() / len(df)).sort_values().to_frame().reset_index().rename(columns={0: 'all'})

In [75]:
proportion_all_piter = proportion_all.merge(proportion_piter)
proportion_all_piter['piter/all'] = proportion_all_piter['piter'] / proportion_all_piter['all']
proportion_all_piter.sort_values(by='piter/all')

,rubric,all,piter,piter/all
1,Спортбар,0.001903,0.000798,0.419485
0,Бар безалкогольных напитков,0.000634,0.000319,0.503383
7,Пиццерия,0.040030,0.023954,0.598402
6,Кальян-бар,0.025091,0.018524,0.738294
2,Кофе с собой,0.008018,0.006547,0.816639
5,Суши-бар,0.023591,0.020441,0.866458
11,Кофейня,0.145296,0.128873,0.886965
3,Кондитерская,0.012344,0.011977,0.970305
10,Быстрое питание,0.134683,0.133504,0.991243
12,Кафе,0.227317,0.226445,0.996164


1) Процент булочных изменился в 1.72 раза

2) Процент спортбаров изменился в 0.42 раза

3) Процент кондитерских, заведений быстрого питания, кафе и ресторанов практически не изменился